In [2]:
import pandas as pd
import numpy as np


## Задания

1)  Разделите данные Титаника (train.csv) на тренировочную, валидационную и тестовую часть. С помощью валидационной части подберите гиперпараметры для моделей Random Forest, XGBoost, Logistic Regression и KNN. Получите точность этих моделей на тестовой части.
2) С помощью RandomForest выберите 2, 4, 8 самых важных признаков и проверьте точность моделей только на этих признаках.

### Task1

#### Разделение данных

In [821]:
data = pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [823]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [825]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [827]:
data_prep = data.copy()

In [829]:
data_prep['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [831]:
data_prep['Embarked'] = data_prep['Embarked'].fillna('S')

In [833]:
data_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [835]:
data_prep['Age'] = data_prep['Age'].fillna(data_prep['Age'].mean())

In [837]:
data_prep = data_prep.drop(['PassengerId', 'Name', 'Ticket' ,'Cabin'], axis = 1)

In [839]:
data_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [841]:
assert not data_prep.isnull().values.any()

In [843]:
data_prep_num = data_prep.drop(['Embarked', 'Sex'], axis=1)
data_prep_emb = pd.get_dummies(data_prep['Embarked'], prefix='Emb')
data_prep_sex = pd.get_dummies(data_prep['Sex'], prefix='Sex')

data_prep = pd.concat((data_prep_num, data_prep_emb, data_prep_sex), axis = 1)


In [845]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(data_prep)
data_prep =  pd.DataFrame(scaled_x, columns=data_prep.columns)

In [847]:
val_data_prep = data_prep['Survived']
data_prep = data_prep.drop(['Survived'], axis = 1)


In [849]:
from sklearn.model_selection import train_test_split

In [851]:
X_temp, X_test, y_temp, y_test = train_test_split(data_prep, val_data_prep, test_size=0.2, random_state=42, stratify=val_data_prep)

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

### Подбор гиперпараметров

#### KNeighborsClassifier

In [855]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [857]:
best_accuracy = 0
best_n_neighbors = None

In [859]:
for n in range(1,40) :
    knn = KNeighborsClassifier(n_neighbors = n)
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_val)
    acc_score = accuracy_score(predicted, y_val)
    if (best_accuracy < acc_score):
        best_accuracy = acc_score
        best_n_neighbors = n 


In [860]:
best_n_neighbors

15

In [863]:
best_knn_model = KNeighborsClassifier(n_neighbors = best_n_neighbors)
best_knn_model.fit(X_train, y_train)
predicted = best_knn_model.predict(X_test)
acc_score = accuracy_score(predicted, y_test)

In [865]:
print(f"best_n_neighbors: {best_n_neighbors}")
print(f"accuracy: {acc_score}")

best_n_neighbors: 15
accuracy: 0.7877094972067039


#### RandomForest

In [868]:
from sklearn.ensemble import RandomForestClassifier

In [870]:
best_accuracy_rf = 0
best_params_rf = None

In [872]:
n_estimators_values = [300, 350, 400]
max_depth_values = [12, 15, 20]
max_features = [0.5, 0.3, 0.2, 0.4]

In [874]:
for n in n_estimators_values:
    for m in max_depth_values:
        for mf in max_features:
            rf = RandomForestClassifier(n_estimators=n, max_depth=m, max_features = mf, random_state=42)
            rf.fit(X_train, y_train)
            predicted = rf.predict(X_val)
            acc_score = accuracy_score(y_val, predicted)
            if best_accuracy_rf < acc_score:
                best_accuracy = acc_score
                best_params_rf = (n, m, mf)

In [875]:
best_rf_model = RandomForestClassifier(n_estimators=best_params_rf[0], max_depth=best_params_rf[1], random_state=42)
best_rf_model.fit(X_train, y_train)
predicted = best_rf_model.predict(X_test)
acc_score = accuracy_score(predicted, y_test)

In [876]:
print(f"best_paramss: {best_params_rf}")
print(f"accuracy: {acc_score}")

best_paramss: (400, 20, 0.4)
accuracy: 0.8044692737430168


### Task2

In [703]:
import matplotlib.pyplot as plt

In [704]:
importances = best_rf_model.feature_importances_

feature_importances = pd.DataFrame(importances, index=data_prep.columns, columns=['importance']).sort_values('importance', ascending=False)

print(feature_importances)

top_n = 2
top_features = feature_importances.head(top_n).index.tolist()
print(f"Наиболее важные признаки: {top_features}")


            importance
Fare          0.260144
Age           0.259796
Sex_male      0.145374
Sex_female    0.128412
Pclass        0.087410
SibSp         0.043532
Parch         0.038909
Emb_S         0.015841
Emb_C         0.011467
Emb_Q         0.009114
Наиболее важные признаки: ['Fare', 'Age']


In [705]:
X_train_2 = X_train[['Fare', 'Sex_male']]
X_test_2 = X_test[['Fare', 'Sex_male']]

In [706]:
best_rf_model = RandomForestClassifier(n_estimators = 80, max_depth=7, random_state=42)

In [707]:
best_rf_model.fit(X_train_2, y_train)

RandomForestClassifier(max_depth=7, n_estimators=80, random_state=42)

In [708]:
predicted = best_rf_model.predict(X_test_2)
acc_score = accuracy_score(predicted, y_test)

In [709]:
print(f"accuracy: {acc_score}")

accuracy: 0.7988826815642458


In [724]:
X_train_4 = X_train[['Fare', 'Age', 'Sex_male', 'Pclass']]
X_test_4 = X_test[['Fare', 'Age', 'Sex_male', 'Pclass']]

In [762]:
best_rf_model = RandomForestClassifier(n_estimators = 100, max_depth=10, random_state=42)

In [764]:
best_rf_model.fit(X_train_4, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [765]:
predicted = best_rf_model.predict(X_test_4)
acc_score = accuracy_score(predicted, y_test)

In [768]:
print(f"accuracy: {acc_score}")

accuracy: 0.8268156424581006


#### XGBoost

In [210]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [212]:
param_grid = {
    'n_estimators': [10, 20, 30, 40, 50, 100, 200, 300], 
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3], 
    'max_depth': [3, 5, 7, 10, 15, 20],  
   }

In [214]:
 model = XGBClassifier()

In [216]:
 grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,  scoring='accuracy')

In [218]:
grid_search.fit(X_train, y_train)

C:\Users\Ilya\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
                         'max_depth': [3, 5, 7, 10, 15, 20],
                         'n_estimators': [10, 20, 30, 40, 50, 100, 200, 300]},
             scoring='accuracy')

In [219]:
best_params_xgb = grid_search.best_params_
model = XGBClassifier(**best_params_xgb)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
acc_score = accuracy_score(predicted, y_test)

In [363]:
params = {
    'booster': 'gbtree',
    'learning_rate': 0.1,
    'n_estimators': 120,
    'max_depth': 6,
    'min_child_weight': 2,
    'subsample': 0.8,
    'colsample_bytree': 0.75,
    'scale_pos_weight': 1,
}
model = XGBClassifier(**params)

model.fit(X_train, y_train)
predicted = model.predict(X_test)
acc_score = accuracy_score(predicted, y_test)

In [365]:
print(f"accuracy: {acc_score}")

accuracy: 0.8268156424581006


#### Logistic Regression

In [368]:
from sklearn.linear_model import LogisticRegression

In [372]:
model = LogisticRegression()

In [374]:
param_grid = {
    'penalty': ['l1', 'l2'], 
    'C': [0.01, 0.1, 1, 10, 100],  
    'max_iter': [100, 200, 300],  
    'solver': ['liblinear', 'saga'],
}

In [376]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)


C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Ilya\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was re

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 300], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [377]:
print(f"best_paramss:{grid_search.best_params_}")
print(f"accuracy:{grid_search.best_score_}")

best_paramss:{'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
accuracy:0.8032445776758947


## Лаборатнорная

1) Загрузить файл, разделить его на train и test. Для test взять 10% случайно выбранных строк таблицы.
2) Обучить модели: Decision Tree, XGBoost, Logistic Regression из библиотек sklearn и xgboost. Обучить модели предсказывать столбец label по остальным столбцам таблицы.
3) Наладить замер Accuracy - доли верно угаданных ответов.
4) Точности всех моделей не должны быть ниже 85%
5) С помощью Decision Tree выбрать 2 самых важных признака и проверить точность модели, обученной только на них.

In [775]:
data = pd.read_csv("titanic_prepared.csv")

In [777]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6634 entries, 0 to 6633
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    6634 non-null   int64  
 1   sex           6634 non-null   int64  
 2   row_number    6634 non-null   float64
 3   liters_drunk  6634 non-null   float64
 4   drink         6634 non-null   int64  
 5   check_number  6634 non-null   int64  
 6   label         6634 non-null   int64  
 7   age_child     6634 non-null   bool   
 8   age_adult     6634 non-null   bool   
 9   age_old       6634 non-null   bool   
 10  morning       6634 non-null   bool   
 11  day           6634 non-null   bool   
 12  evening       6634 non-null   bool   
dtypes: bool(6), float64(2), int64(5)
memory usage: 401.8 KB


In [779]:
data

,Unnamed: 0,sex,row_number,liters_drunk,drink,check_number,label,age_child,age_adult,age_old,morning,day,evening
0,0,0,90.0,1.0,1,8092,1,False,False,True,False,False,True
1,1,0,81.0,4.0,1,4252,1,False,True,False,False,True,False
2,2,1,14.0,3.0,0,6913,1,False,False,True,False,False,True
3,3,1,0.0,1.0,0,8479,1,False,False,True,False,True,False
4,4,1,84.0,1.0,0,6279,0,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6629,6629,1,55.0,2.0,0,1526,1,False,False,True,False,False,True
6630,6630,1,75.0,5.0,1,6690,0,False,False,True,False,True,False
6631,6631,1,1.0,2.0,1,2834,0,False,True,False,True,False,False
6632,6632,1,68.0,4.0,0,5126,1,False,True,False,False,True,False


In [785]:
data= data.drop(['Unnamed: 0'], axis = 1)
data

,sex,row_number,liters_drunk,drink,check_number,label,age_child,age_adult,age_old,morning,day,evening
0,0,90.0,1.0,1,8092,1,False,False,True,False,False,True
1,0,81.0,4.0,1,4252,1,False,True,False,False,True,False
2,1,14.0,3.0,0,6913,1,False,False,True,False,False,True
3,1,0.0,1.0,0,8479,1,False,False,True,False,True,False
4,1,84.0,1.0,0,6279,0,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
6629,1,55.0,2.0,0,1526,1,False,False,True,False,False,True
6630,1,75.0,5.0,1,6690,0,False,False,True,False,True,False
6631,1,1.0,2.0,1,2834,0,False,True,False,True,False,False
6632,1,68.0,4.0,0,5126,1,False,True,False,False,True,False


In [787]:
X_prep = data.drop(['label'], axis = 1)
Y_prep = data['label']

In [805]:
X_prep['age_child'].value_counts()

age_child
False    5911
True      723
Name: count, dtype: int64

In [809]:
X_prep['age_child'] = X_prep['age_child'].apply(lambda x: 1 if x else 0)
X_prep['age_adult'] = X_prep['age_adult'].apply(lambda x: 1 if x else 0)
X_prep['age_old'] = X_prep['age_old'].apply(lambda x: 1 if x else 0)
X_prep['morning'] = X_prep['morning'].apply(lambda x: 1 if x else 0)
X_prep['day'] = X_prep['day'].apply(lambda x: 1 if x else 0)
X_prep['evening'] = X_prep['evening'].apply(lambda x: 1 if x else 0)


In [811]:
X_prep

,sex,row_number,liters_drunk,drink,check_number,age_child,age_adult,age_old,morning,day,evening
0,0,90.0,1.0,1,8092,0,0,1,0,0,1
1,0,81.0,4.0,1,4252,0,1,0,0,1,0
2,1,14.0,3.0,0,6913,0,0,1,0,0,1
3,1,0.0,1.0,0,8479,0,0,1,0,1,0
4,1,84.0,1.0,0,6279,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6629,1,55.0,2.0,0,1526,0,0,1,0,0,1
6630,1,75.0,5.0,1,6690,0,0,1,0,1,0
6631,1,1.0,2.0,1,2834,0,1,0,1,0,0
6632,1,68.0,4.0,0,5126,0,1,0,0,1,0


In [881]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_prep_np = scaler.fit_transform(X_prep)
X_prep =  pd.DataFrame(X_prep_np, columns=X_prep.columns)
X_prep

,sex,row_number,liters_drunk,drink,check_number,age_child,age_adult,age_old,morning,day,evening
0,0.0,0.90,0.2,1.0,0.809281,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.81,0.8,1.0,0.425243,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,0.14,0.6,0.0,0.691369,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.00,0.2,0.0,0.847985,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.84,0.2,0.0,0.627963,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
6629,1.0,0.55,0.4,0.0,0.152615,0.0,0.0,1.0,0.0,0.0,1.0
6630,1.0,0.75,1.0,1.0,0.669067,0.0,0.0,1.0,0.0,1.0,0.0
6631,1.0,0.01,0.4,1.0,0.283428,0.0,1.0,0.0,1.0,0.0,0.0
6632,1.0,0.68,0.8,0.0,0.512651,0.0,1.0,0.0,0.0,1.0,0.0


In [889]:
X_train, X_test, y_train, y_test = train_test_split(X_prep, Y_prep, test_size=0.1, random_state=42, stratify=Y_prep)

### DecisionTree

In [885]:
from sklearn.tree import DecisionTreeClassifier

In [893]:
model_dt = DecisionTreeClassifier(max_depth=3, criterion='entropy')

In [895]:
model_dt.fit(X_train, y_train)
predicted = model_dt.predict(X_test)
acc_score_dt = accuracy_score(predicted, y_test)
print(f"accuracy: {acc_score_dt}")

accuracy: 0.8614457831325302


In [900]:
importances = model_dt.feature_importances_

feature_importances = pd.DataFrame(importances, index=X_prep.columns, columns=['importance']).sort_values('importance', ascending=False)

print(feature_importances)

top_n = 2
top_features = feature_importances.head(top_n).index.tolist()
print(f"Наиболее важные признаки: {top_features}")

              importance
morning         0.716598
day             0.167985
liters_drunk    0.067425
age_old         0.047992
sex             0.000000
row_number      0.000000
drink           0.000000
check_number    0.000000
age_child       0.000000
age_adult       0.000000
evening         0.000000
Наиболее важные признаки: ['morning', 'day']


In [918]:
X_prep_drop = X_prep.drop(['sex', 'row_number', 'drink', 'check_number', 'age_child', 'age_adult', 'evening', 'liters_drunk', 'age_old'], axis = 1)
X_train_drop, X_test_drop, y_train, y_test = train_test_split(X_prep_drop, Y_prep, test_size=0.1, random_state=42, stratify=Y_prep)

In [920]:
model_dt_2 = DecisionTreeClassifier(max_depth=3, criterion='entropy')
model_dt_2.fit(X_train_drop, y_train)
predicted = model_dt_2.predict(X_test_drop)
acc_score_dt_dr = accuracy_score(predicted, y_test)
print(f"accuracy top 2 feature: {acc_score_dt_dr}")

accuracy top 2 feature: 0.8614457831325302


### XGBoost

In [925]:
params = {
    'booster': 'gbtree',
    'learning_rate': 0.1,
    'n_estimators': 120,
    'max_depth': 6,
    'min_child_weight': 2,
    'subsample': 0.8,
    'colsample_bytree': 0.75,
    'scale_pos_weight': 1,
}
model_xgb = XGBClassifier(**params)

model_xgb.fit(X_train, y_train)
predicted = model_xgb.predict(X_test)
acc_score_xgb = accuracy_score(predicted, y_test)
print(f"accuracy: {acc_score_xgb}")

accuracy: 0.8840361445783133


### Logistic Regression

In [934]:
model_lr = LogisticRegression(C=10, max_iter=100,penalty='l1', solver='liblinear')
model_lr.fit(X_train, y_train)
predicted = model_lr.predict(X_test)
acc_score_lr = accuracy_score(predicted, y_test)
print(f"accuracy: {acc_score_lr}")

accuracy: 0.8614457831325302
